# Adversarial Training with `mister_ed`
This file will contain the basics on how to perform adversarial training under the `mister_ed` framework. It's highly recommended that you have walked through tutorial_1 before going through this.

As usual, we'll start by importing everything we'll need

In [12]:
# EXTERNAL LIBRARY IMPORTS
import numpy as np 
import scipy 
import matplotlib.pyplot as plt 

import torch # Need torch version >=0.3
import torch.nn as nn 
import torch.optim as optim 
assert float(torch.__version__[:3]) >= 0.3


In [13]:
# MISTER ED SPECIFIC IMPORT BLOCK
# (here we do things so relative imports work )
# Universal import block 
# Block to get the relative imports working 
import os
import sys 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)


import config
import prebuilt_loss_functions as plf
import loss_functions as lf 
import utils.pytorch_utils as utils
import utils.image_utils as img_utils
import cifar10.cifar_loader as cifar_loader
import cifar10.cifar_resnets as cifar_resnets
import adversarial_training as advtrain
import adversarial_evaluation as adveval
import utils.checkpoints as checkpoints
import adversarial_perturbations as ap 
import adversarial_attacks as aa
import spatial_transformers as st


Now let's define what we want to do here:

Our goal is to run through a few training epochs of a pretrained classifier where we augment the training data with a set of adversarial examples. For simplicity's sake, let's just try and train a few epochs of a 20-layer ResNet trained on CIFAR-10, defended against an FGSM attack of $\epsilon=8$.

First things first, let's instatiate our pretrained classifier and our training dataset. 

In [14]:
cifar_trainset = cifar_loader.load_cifar_data('train')
model, normalizer = cifar_loader.load_pretrained_cifar_resnet(flavor=20, return_normalizer=True)

Files already downloaded and verified


And now let's build the attack parameters: an object that contains all the information to perform an attack on a minibatch. So first let's build an attack object and then furnish it with the necessary kwargs. 

Like in tutorial 1, to create an attack object, we'll need to create a threat model and a loss function

In [15]:
delta_threat = ap.ThreatModel(ap.DeltaAddition, 
                              {'lp_style': 'inf', 
                               'lp_bound': 8.0 / 255})
attack_loss = plf.VanillaXentropy(model, normalizer)
attack_object = aa.FGSM(model, normalizer, delta_threat, attack_loss)


And then we build the `AttackParameters` object, which just wraps the attack object with the kwargs needed to call the `attack(...)` method on attack. For FGSM attacks, we just want to turn the verbosity off, but for more complicated attacks, this will be more involved. Typically in training, we generate a single adversarial example per training point, but to be speedy here, let's only create 1 example per every 5 training points.


In [16]:
attack_kwargs = {'verbose': False} # kwargs to be called in attack_object.attack(...)
attack_params = advtrain.AdversarialAttackParameters(attack_object, proportion_attacked=0.2, 
                                                     attack_specific_params={'attack_kwargs': attack_kwargs})


With our attack parameters built, we can build the object that handles training for us: this is instatiated with knowledge of the classifier, normalizer and some identifying features such as the *name* of the experiment and architecture. It's worthwhile to be informative with these so you keep which attacks this model is trained against straight.

In [17]:
experiment_name = 'tutorial_fgsm'
architecture = 'resnet20'
training_obj = advtrain.AdversarialTraining(model, normalizer, experiment_name, architecture)

When you start training though, you'll need to furnish the trainer with some extra arguments:
    - the data loader 
    - the number of epochs to train for 
    - a loss function (not one of the `mister_ed` custom loss functions though!)
    - which optimizer to use (defaults to Adam with decent hyperparams)
    - the attack parameters object 
    - whether or not to use the gpu (defaults to not using GPU)
    - the verbosity level (ranging from ['low', 'medium', 'high', 'snoop'] (defaults to 'medium')
    - whether or not to save the generated adversarial examples as images (defaults to false)
    
To be cute, we'll just train for two epochs so you get the picture. Also note that unless the verbosity is set to `low`, a checkpoint will be saved after every epoch. By default, these checkpoints are named like `<experiment_name>.<architecture_name>.<epoch>.path.tar`


In [ ]:
train_loss = nn.CrossEntropyLoss() # just use standard XEntropy to train
training_logger = training_obj.train(cifar_trainset, 2, train_loss, 
                                     attack_parameters=attack_params, 
                                     verbosity='snoop', loglevel='snoop') 

[1,     1] accuracy: (48.000, 96.000)
[1,     1] loss: 0.359370
[1,     2] accuracy: (40.000, 100.000)
[1,     2] loss: 0.308328
[1,     3] accuracy: (24.000, 88.000)
[1,     3] loss: 0.370927
[1,     4] accuracy: (24.000, 100.000)
[1,     4] loss: 0.317369
[1,     5] accuracy: (36.000, 100.000)
[1,     5] loss: 0.351738
[1,     6] accuracy: (44.000, 88.000)
[1,     6] loss: 0.318250
[1,     7] accuracy: (44.000, 100.000)
[1,     7] loss: 0.273458
[1,     8] accuracy: (44.000, 100.000)
[1,     8] loss: 0.341295
[1,     9] accuracy: (36.000, 96.000)
[1,     9] loss: 0.403555
[1,    10] accuracy: (32.000, 92.000)
[1,    10] loss: 0.450216
[1,    11] accuracy: (56.000, 100.000)
[1,    11] loss: 0.354675
[1,    12] accuracy: (48.000, 96.000)
[1,    12] loss: 0.338677
[1,    13] accuracy: (36.000, 100.000)
[1,    13] loss: 0.381816
[1,    14] accuracy: (44.000, 96.000)
[1,    14] loss: 0.338495
[1,    15] accuracy: (44.000, 100.000)
[1,    15] loss: 0.296192
[1,    16] accuracy: (36.000, 92

[1,   128] loss: 0.324646
[1,   129] accuracy: (36.000, 92.000)
[1,   129] loss: 0.453300
[1,   130] accuracy: (52.000, 96.000)
[1,   130] loss: 0.274612
[1,   131] accuracy: (56.000, 100.000)
[1,   131] loss: 0.299975
[1,   132] accuracy: (56.000, 92.000)
[1,   132] loss: 0.331249
[1,   133] accuracy: (28.000, 92.000)
[1,   133] loss: 0.338712
[1,   134] accuracy: (40.000, 96.000)
[1,   134] loss: 0.340625
[1,   135] accuracy: (52.000, 96.000)
[1,   135] loss: 0.380498
[1,   136] accuracy: (40.000, 88.000)
[1,   136] loss: 0.420739
[1,   137] accuracy: (56.000, 100.000)
[1,   137] loss: 0.337408
[1,   138] accuracy: (56.000, 96.000)
[1,   138] loss: 0.326675
[1,   139] accuracy: (32.000, 92.000)
[1,   139] loss: 0.417188
[1,   140] accuracy: (36.000, 92.000)
[1,   140] loss: 0.340222
[1,   141] accuracy: (72.000, 100.000)
[1,   141] loss: 0.351880
[1,   142] accuracy: (56.000, 100.000)
[1,   142] loss: 0.372170
[1,   143] accuracy: (32.000, 100.000)
[1,   143] loss: 0.327759
[1,   144

[1,   256] loss: 0.312985
[1,   257] accuracy: (40.000, 100.000)
[1,   257] loss: 0.318875
[1,   258] accuracy: (32.000, 100.000)
[1,   258] loss: 0.334218
[1,   259] accuracy: (36.000, 92.000)
[1,   259] loss: 0.410454
[1,   260] accuracy: (48.000, 96.000)
[1,   260] loss: 0.327219
[1,   261] accuracy: (56.000, 96.000)
[1,   261] loss: 0.314524
[1,   262] accuracy: (44.000, 96.000)
[1,   262] loss: 0.442953
[1,   263] accuracy: (52.000, 88.000)
[1,   263] loss: 0.436650
[1,   264] accuracy: (40.000, 92.000)
[1,   264] loss: 0.319648
[1,   265] accuracy: (60.000, 100.000)
[1,   265] loss: 0.296856
[1,   266] accuracy: (48.000, 96.000)
[1,   266] loss: 0.321136
[1,   267] accuracy: (44.000, 96.000)
[1,   267] loss: 0.476226
[1,   268] accuracy: (44.000, 92.000)
[1,   268] loss: 0.393263
[1,   269] accuracy: (36.000, 100.000)
[1,   269] loss: 0.299720
[1,   270] accuracy: (56.000, 100.000)
[1,   270] loss: 0.269755
[1,   271] accuracy: (48.000, 92.000)
[1,   271] loss: 0.362542
[1,   272

[1,   388] accuracy: (44.000, 96.000)
[1,   388] loss: 0.389356
[1,   389] accuracy: (48.000, 96.000)
[1,   389] loss: 0.350460
[1,   390] accuracy: (24.000, 88.000)
[1,   390] loss: 0.431381
[1,   391] accuracy: (37.500, 100.000)
[1,   391] loss: 0.326515
COMPLETED EPOCH 0001... checkpointing here
[2,     1] accuracy: (40.000, 100.000)
[2,     1] loss: 0.352001
[2,     2] accuracy: (52.000, 100.000)
[2,     2] loss: 0.255962
[2,     3] accuracy: (40.000, 88.000)
[2,     3] loss: 0.378638
[2,     4] accuracy: (60.000, 96.000)
[2,     4] loss: 0.295902
[2,     5] accuracy: (28.000, 96.000)
[2,     5] loss: 0.364539
[2,     6] accuracy: (40.000, 92.000)
[2,     6] loss: 0.297084
[2,     7] accuracy: (44.000, 92.000)
[2,     7] loss: 0.330333
[2,     8] accuracy: (56.000, 96.000)
[2,     8] loss: 0.417643
[2,     9] accuracy: (36.000, 92.000)
[2,     9] loss: 0.377433
[2,    10] accuracy: (52.000, 100.000)
[2,    10] loss: 0.321050
[2,    11] accuracy: (44.000, 96.000)
[2,    11] loss: 0.

[2,   128] accuracy: (56.000, 100.000)
[2,   128] loss: 0.270638
[2,   129] accuracy: (36.000, 100.000)
[2,   129] loss: 0.329888
[2,   130] accuracy: (24.000, 96.000)
[2,   130] loss: 0.318928
[2,   131] accuracy: (28.000, 100.000)
[2,   131] loss: 0.352655
[2,   132] accuracy: (64.000, 96.000)
[2,   132] loss: 0.336952
[2,   133] accuracy: (40.000, 88.000)
[2,   133] loss: 0.398387
[2,   134] accuracy: (56.000, 92.000)
[2,   134] loss: 0.375803
[2,   135] accuracy: (28.000, 92.000)
[2,   135] loss: 0.401924
[2,   136] accuracy: (44.000, 100.000)
[2,   136] loss: 0.376128
[2,   137] accuracy: (56.000, 96.000)
[2,   137] loss: 0.296430
[2,   138] accuracy: (56.000, 100.000)
[2,   138] loss: 0.316026
[2,   139] accuracy: (48.000, 92.000)
[2,   139] loss: 0.386375
[2,   140] accuracy: (44.000, 96.000)
[2,   140] loss: 0.325530
[2,   141] accuracy: (64.000, 96.000)
[2,   141] loss: 0.272427
[2,   142] accuracy: (44.000, 88.000)
[2,   142] loss: 0.393410
[2,   143] accuracy: (56.000, 88.00

The printouts look like:
``` 
[epoch_no, minibatch_no] accuracy: (X, Y) 
[epoch_no, minibatch_no] loss: Z
```

- X is the percent of successfully classified *adversarial* examples generated from that minibatch only
- Y is the percent of successfully classified *original* examples on that minibatch only
- Z is the value of the loss function after that minibatch

The output of training is a `TrainingLogger` class that stores essentially what was printed, and is separately controlled by its own `loglevel` argument. This can be accessed (and just to be safe, sorted using `sort_series`) and plotted as follows:

In [ ]:
# Plot training loss
import matplotlib.pyplot as plt 
plt.plot(training_logger.sort_series('training_loss', return_keys=False))

In [ ]:
# Plot adversarial accuracy as we train 
# This should be noisy, but generally going UP
plt.plot([_[0] for _ in training_logger.sort_series('attack', return_keys=False)])

Once training completes, you can verify that the checkpoints are indeed stored in wherever you have set up pretrained models to be stored. By default this is `mister_ed/pretrained_models/`, so you should have a `tutorial_fgsm.resnet20.000002.path.tar` file

# Restarting from Checkpoint
When training, sometimes @#\$& happens and things break. This is why we checkpoint. Here we'll show how to restart from checkpoint in training:

Suppose we want to pick back up from where we left off with the experiment/architecture pair defined above `(tutorial_fgsm, resnet20)`. Then we want to do the following steps:

1. Instantiate a model of the same architecture (weights don't matter, since we'll load from the checkpoint) 
2. Build an `AdversarialTraining` object using this model, its normalizer, and the same experiment name, architecture name 
3. Build a loss function, attack_parameters object, and all other identical kwargs from the first (aborted) training run 
4. Run the training using the training object's `train_from_checkpoint` method instead of `train`. All the kwargs are the same 

In [ ]:
naive_model, normalizer = cifar_loader.load_pretrained_cifar_resnet(flavor=20, return_normalizer=True)
new_train_obj = advtrain.AdversarialTraining(naive_model, normalizer, experiment_name, architecture)

delta_threat = ap.ThreatModel(ap.DeltaAddition, 
                              {'lp_style': 'inf', 
                               'lp_bound': 8.0 / 255})
attack_loss = plf.VanillaXentropy(naive_model, normalizer)
attack_object = aa.FGSM(naive_model, normalizer, delta_threat, attack_loss)
attack_kwargs = {'verbose': False} # kwargs to be called in attack_object.attack(...)
attack_params = advtrain.AdversarialAttackParameters(attack_object, proportion_attacked=0.2, 
                                                     attack_specific_params={'attack_kwargs': attack_kwargs})

new_train_obj.train_from_checkpoint(cifar_trainset, 4, train_loss, attack_parameters=attack_params, 
                                    verbosity='high')

Once this finishes, notice that you should now have a file 
`tutorial_fgsm.resnet20.000004.path.tar` in your pretrained_models directory.

# Using the training script 
Using an ipython notebook isn't typically ideal for training, since it mandates you keep your browser window open. To this end, we've built a script to perform adversarial training in a tmux/screen background. This is located in `scripts/advtrain.py`. Here's what we've found are best practices for doing this:

- Copy `scripts/advtrain.py` into `scripts/advtrain_<DESCRIPTIVE_EXPERIMENT_NAME>.py`
- Modify the `build_attack_params` method in `scripts/advtrain_<DESCRIPTIVE_EXPERIMENT_NAME>.py` to use the attack parameters that you want. There's plenty of prebuilt attack parameters in that file to choose from. 
- In a tmux/screen, from `mister_ed`, run 

```python -m scripts.advtrain_DESCRIPTIVE_EXPERIMENT_NAME --exp <DESCRIPTIVE_EXPERIMENT_NAME> --arch <ARCHITECTURE_CHOICE> --verbosity [snoop/high/medium]```

- To resume, you can optionally add the `-r` or `--resume` flag to the script call


# A note on GPU Usage
If you have access to a GPU on your machine, you'll probably want to leverage its power when doing training and attacks. `mister_ed` has been designed so "standard" GPU behavior should be supported without any extra effort. By "standard" GPU behavior, I mean that all objects reside on the same device: either all on the GPU or none on the GPU. If there is a GPU on your machine, which one can check from the output of 
```
import torch.cuda as cuda 
print(cuda.is_available()) 
```
Globally, unless otherwise specified, all objects will be initialized in GPU-mode if this output is `True`. This is done behind-the-scenes by setting the environment variable `MISTER_ED_GPU`. If you have access to a GPU, but wouldn't like to use it, you can manually override this environment variable by calling:
```
import utils.pytorch_utils as utils 
utils.set_global_gpu(False)
```
And then none of your objects will be in GPU-mode by default. 

For nonstandard GPU behavior, you should initialize any object that differs from the default gpu status (as defined by `MISTER_ED_GPU`) with the kwarg `manual_gpu=<True/False>`
